In [6]:
import csv 
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import csv 
def get_data(days_delta = 1, threshold = 0.005 ):

    
    


    target = dict()
    file_market = open('market_ready.csv', 'r')
    reader_market = csv.DictReader(file_market)
    market_date_list = []


    for a_row in reader_market:
        date = datetime.strptime( a_row['date'], "%Y-%m-%d").date()
        target[date]  = [-1]*5
        r1 = float(a_row[u'1日報酬'])
        r2 = float(a_row[u'2日報酬'])
        r3 = float(a_row[u'3日報酬'])
        r4 = float(a_row[u'4日報酬'])
        r5 = float(a_row[u'5日報酬'])
        return_list = [r1,r2,r3,r4,r5]
        

        market_date_list.append(date)
        for i in range(5):
            return_i = return_list[i]
            if return_i < -1 * threshold : 
                target[date][i] = 0
            elif return_i > threshold :
                target[date][i] = 2
            else:
                target[date][i] = 1
    
    
    
    time_delta = timedelta(hours = -13.5)
    
    df_list = []
    for file_name in ['news_labeled.csv', 'bbs_labeled.csv', 'forum_labeled.csv']:
        dict_for_number_of_up_and_down = dict() # 兩個值, 第一個代表label的漲跌相差數，第二個值代表總文章數
        
        if file_name.split('_')[0]=='news':
            coding = 'big5'
        else:
            coding = 'utf-8'
        with open( file_name, encoding = coding ) as f:

            csv_reader = csv.DictReader(f)
            for a_row in csv_reader:
                if coding == 'utf-8':
                    time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M:%S") 
                else:
                    time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M") 
                adj_time = time+time_delta
                adj_date = (adj_time + timedelta(days = days_delta )).date()
                label = float(a_row['label'])
                #cluster = a_row['cluster']


                if  adj_date in market_date_list: 

                    try:
                        dict_for_number_of_up_and_down[adj_date][0] += (label - 1)
                        dict_for_number_of_up_and_down[adj_date][1] += 1  

                    except:
                        dict_for_number_of_up_and_down[adj_date] = [0, 0]
        #standarize
        standardize_value = dict()
        for a_list, dates in zip(  dict_for_number_of_up_and_down.values(), dict_for_number_of_up_and_down.keys()   ): 
            if a_list[1] == 0:
                standardize_value[dates] = [ 0., target[dates][days_delta-1]  ]
            else:
                standardize_value[dates] = [  a_list[0]/ a_list[1], target[dates][days_delta-1]  ]

        #get df
        df = pd.DataFrame.from_dict(standardize_value).T
        df.columns = [ file_name.split('_')[0], 'target%i' %days_delta ]
        df_list.append(df)
        
    df = pd.concat([df_list[1]['bbs'], df_list[0]], axis = 1, join='inner')
    df = pd.concat([df_list[2]['forum'], df], axis = 1, join='inner')
    return df.sort_index()

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score 
max_score = 0
for i in range(1,6):
    for j in np.linspace(0, 0.005, num=6):
        df_n_b_f = get_data(i, float(j) )

        # RF
        X = df_n_b_f.iloc[:,0:3]
        y = df_n_b_f['target%i' %i]

        clf =  RandomForestClassifier(criterion = 'entropy', n_estimators =12, random_state = 1, n_jobs = 4)

        # scores

        scores = cross_val_score(clf, X, y, cv= 6, scoring='f1_macro')
        
        print(i, '%0.3f' %float(j) , scores.mean())
        
        
        if scores.mean() >  max_score:
            max_score = scores.mean()
            max_combination = ( i,float(j) )
            
        


1 0.000 0.5140363989573898
1 0.001 0.3493334742705691
1 0.002 0.36500277883244164
1 0.003 0.31407975170251534
1 0.004 0.30046125174734667
1 0.005 0.3258464328407081
2 0.000 0.5568905363247105
2 0.001 0.329981787951565
2 0.002 0.35069178008373864
2 0.003 0.3355704473337994
2 0.004 0.3299482939568132
2 0.005 0.3496629230655863
3 0.000 0.460760613645804
3 0.001 0.3235683834193267
3 0.002 0.31514271107050545
3 0.003 0.2950746356772132
3 0.004 0.34262457596051993
3 0.005 0.3131642489434792
4 0.000 0.47727594029575643
4 0.001 0.3365049913803078
4 0.002 0.30890155428991917
4 0.003 0.32400103414491777
4 0.004 0.3152529099554286
4 0.005 0.3405550984789663
5 0.000 0.48497183814655603


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


5 0.001 0.31153004352748476
5 0.002 0.2962837437589964
5 0.003 0.2915799478891708
5 0.004 0.31203712639424563
5 0.005 0.34188295503080307


In [16]:
i, j = (1,0)
df_n_b_f = get_data(i, float(j) )

X = df_n_b_f.iloc[:,0:3]
y = df_n_b_f['target%i' %i]
#split
from sklearn.model_selection import train_test_split as tt_split
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.2,random_state=0)

clf =  RandomForestClassifier(criterion = 'entropy', n_estimators =5, random_state = 1, n_jobs = 2)

#fit
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
from sklearn.metrics import classification_report
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.41      0.34      0.37        62
     class 2       0.56      0.63      0.59        82

   micro avg       0.51      0.51      0.51       144
   macro avg       0.49      0.49      0.48       144
weighted avg       0.50      0.51      0.50       144



In [44]:
i, j = (2,0)
df_n_b_f = get_data(i, float(j) )

X = df_n_b_f.iloc[:,0:3]
y = df_n_b_f['target%i' %i]
#split
from sklearn.model_selection import train_test_split as tt_split
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.2,random_state=0)

clf =  RandomForestClassifier(criterion = 'entropy', n_estimators =5, random_state = 1, n_jobs = 2)

#fit
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
from sklearn.metrics import classification_report
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.55      0.42      0.47        72
     class 2       0.53      0.66      0.59        73

   micro avg       0.54      0.54      0.54       145
   macro avg       0.54      0.54      0.53       145
weighted avg       0.54      0.54      0.53       145



# 接下來我需要找到最好的分類器並超參優化

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score 
base_score = 0.55689
max_score = 0
df_n_b_f = get_data(2, 0 )
X = df_n_b_f.iloc[:,0:3]
y = df_n_b_f['target%i' %i]
for n in np.linspace(10, 100, num=91):
    clf =  RandomForestClassifier(criterion = 'entropy', n_estimators =int(n), random_state = 1, n_jobs = 4)

    # scores

    scores = cross_val_score(clf, X, y, cv= 6, scoring='f1_macro')

    print(i, '%0.3f' %float(j) , scores.mean())


    if scores.mean() >  max_score:
        
        max_score = scores.mean()
        max_combination = ( i,float(j) )
        print('Improve %0.4f score' %(max_score-base_score) )
            
        


2 0.000 0.5468493855547819
Improve -0.0100 score
2 0.000 0.5505138196184192
Improve -0.0064 score
2 0.000 0.5568905363247105
Improve 0.0000 score
2 0.000 0.5528395829233822
2 0.000 0.5604090742855375
Improve 0.0035 score
2 0.000 0.5622043119974011
Improve 0.0053 score
2 0.000 0.5591095552266098
2 0.000 0.5439009048229495
2 0.000 0.5542809896815913
2 0.000 0.5463281658107331
2 0.000 0.5471638636538231
2 0.000 0.5500421880439341
2 0.000 0.5698645346032657
Improve 0.0130 score
2 0.000 0.5549965746648405
2 0.000 0.5509068675786081
2 0.000 0.5482226343783266
2 0.000 0.5610978210978957
2 0.000 0.5546629187103109
2 0.000 0.5571005614105573
2 0.000 0.543183657855261
2 0.000 0.5486486077028379
2 0.000 0.5376309967985496
2 0.000 0.5443983019288486
2 0.000 0.5408957603028156
2 0.000 0.5417982744378043
2 0.000 0.5335205760484185
2 0.000 0.5416849490147722
2 0.000 0.540527484641039
2 0.000 0.5438132758337616
2 0.000 0.5422362394700693
2 0.000 0.5513934252896242
2 0.000 0.5473594633386568
2 0.000 0.

In [28]:
df_n_b_f = get_data(2, 0 )
X = df_n_b_f.iloc[:,0:3]
y = df_n_b_f['target%i' %i]
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.2,random_state=0)

In [40]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 11)]

# criterion to choose tree
criterions = ['gini', 'entropy'] 

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 7, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'criterion' : criterions,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2500, 
                               cv = 6, verbose=2, random_state=42, n_jobs = 4, scoring = 'f1_macro')

# Fit the random search model
rf_random.fit(x_train, y_train)

Fitting 6 folds for each of 2500 candidates, totalling 15000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  41 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 767 tasks      | elapsed:    7.6s
[Parallel(n_jobs=4)]: Done 1985 tasks      | elapsed:   17.3s
[Parallel(n_jobs=4)]: Done 3683 tasks      | elapsed:   30.7s
[Parallel(n_jobs=4)]: Done 5873 tasks      | elapsed:   48.3s
[Parallel(n_jobs=4)]: Done 8543 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 11705 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 15000 out of 15000 | elapsed:  2.0min finished
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=6, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=2500, n_jobs=4,
          param_distributions={'n_estimators': [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30], 'criterion': ['gini', 'entropy'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 4, 6, 8, 10], 'min_samples_leaf': [1, 2, 4, 7, 10], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='f1_macro', verbose=2)

In [41]:
rf_random.best_params_

{'n_estimators': 28,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_depth': 90,
 'criterion': 'entropy',
 'bootstrap': True}

In [51]:
clf =  RandomForestClassifier(criterion = 'entropy', n_estimators = 28, max_depth = 90, min_samples_split = 6, random_state = 42, n_jobs = 4)

scores = cross_val_score(clf, x_train, y_train, cv= 6, scoring='f1_macro')
print(scores.mean())
#fit
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
from sklearn.metrics import classification_report
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

0.5236382858966038
              precision    recall  f1-score   support

     class 0       0.48      0.38      0.42        72
     class 2       0.49      0.60      0.54        73

   micro avg       0.49      0.49      0.49       145
   macro avg       0.49      0.49      0.48       145
weighted avg       0.49      0.49      0.48       145



## 莫名其妙得到很差的結果 不知道怎麼了???
## 我想改提取關鍵字了